In [ ]:
# Logistic Regression (5)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# for resproducibility
torch.manual_seed(1)

In [ ]:
# training data
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [ ]:
# whole training procedure
# 모델 초기화
W = torch.zeros((2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W,b],lr=1)
nb_epochs = 1000
for epoch in range(nb_epochs+1):
  # cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W)+b) # or .nm or @
  cost = F.binary_cross_entropy(hypothesis,y_train)
  # cost로 H(x) 계산
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [ ]:
# Higher Implementation with Class
class BinaryClassifier(nn.Module):
  def __init__ (self):
    super().__init__()
    self.linear = nn.Linear(2,1)
    self.sigmoid = nn.Sigmoid()
  def forward(self,x):
    return self.sigmoid(self.linear(x))
model = BinaryClassifier()
# training data
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
# optimizer 설정
optimizer = optim.SGD(model.parameters(),lr=1)
nb_epochs = 100
for epoch in range(nb_epochs+1):
  # H(x) 계산
  hypothesis = model(x_train)
  # cost 계산
  cost = F.binary_cross_entropy(hypothesis,y_train)
  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  # 10번마다 로그 출력
  if epoch % 10 == 0:
    prediction = hypothesis>=torch.FloatTensor([0.5])
    correct_prediction = prediction.float()==y_train
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print('Epoch {:4d}/{} Cost: {:6f} Accuracy {:2.2f}%'.format(
        epoch,nb_epochs,cost.item(),accuracy*100,
    ))

Epoch    0/100 Cost: 0.944424 Accuracy 50.00%
Epoch   10/100 Cost: 0.583521 Accuracy 83.33%
Epoch   20/100 Cost: 0.559472 Accuracy 83.33%
Epoch   30/100 Cost: 0.458799 Accuracy 83.33%
Epoch   40/100 Cost: 0.373940 Accuracy 83.33%
Epoch   50/100 Cost: 0.294290 Accuracy 83.33%
Epoch   60/100 Cost: 0.224786 Accuracy 83.33%
Epoch   70/100 Cost: 0.176302 Accuracy 100.00%
Epoch   80/100 Cost: 0.152989 Accuracy 100.00%
Epoch   90/100 Cost: 0.141202 Accuracy 100.00%
Epoch  100/100 Cost: 0.131799 Accuracy 100.00%


In [ ]:
# Softmax Classification (6)

In [ ]:
z = torch.FloatTensor([1,2,3])
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
hypothesis.sum()

tensor([0.0900, 0.2447, 0.6652])


tensor(1.)

In [ ]:
z = torch.rand(3,5,requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)
y = torch.randint(5,(3,)).long()
print(y)

tensor([[0.1831, 0.2236, 0.1995, 0.2336, 0.1602],
        [0.1740, 0.1725, 0.1940, 0.1801, 0.2794],
        [0.2618, 0.1860, 0.1656, 0.2060, 0.1806]], grad_fn=<SoftmaxBackward>)
tensor([3, 1, 4])


In [ ]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)
cost = (y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.6410, grad_fn=<MeanBackward0>)


In [ ]:
# training with low-level cross entropy loss (1)
x_train = [[1,2,1,1],[2,1,3,2],[1,2,2,2],[1,3,3,3],[1,4,4,4],[1,5,5,5],[1,6,6,6],[1,7,7,7]]
y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
# 모델 초기화
W = torch.zeros((4,3),requires_grad=True)
b = torch.zeros(1,requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W,b],lr=0.1)
nb_epochs=1000
for epoch in range (nb_epochs+1):
  # Cost 계산
  hypothesis = F.softmax(x_train.matmul(W)+b, dim=1)
  y_one_hot = torch.zeros_like(hypothesis)
  y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
  cost=(y_one_hot*-torch.log(F.softmax(hypothesis,dim=1))).sum(dim=1).mean()
  # Cost로 H(x) 계산
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch,nb_epochs,cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.969450
Epoch  200/1000 Cost: 0.913969
Epoch  300/1000 Cost: 0.879853
Epoch  400/1000 Cost: 0.855244
Epoch  500/1000 Cost: 0.836075
Epoch  600/1000 Cost: 0.820364
Epoch  700/1000 Cost: 0.807046
Epoch  800/1000 Cost: 0.795501
Epoch  900/1000 Cost: 0.785337
Epoch 1000/1000 Cost: 0.776287


In [ ]:
# training with low-level cross entropy loss (2)
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear=nn.Linear(4,3)
  def forward(self,x):
    return self.linear(x)
model = SoftmaxClassifierModel()
x_train = [[1,2,1,1],[2,1,3,2],[1,2,2,2],[1,3,3,3],[1,4,4,4],[1,5,5,5],[1,6,6,6],[1,7,7,7]]
y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
# 모델 초기화
W = torch.zeros((4,3),requires_grad=True)
b = torch.zeros(1,requires_grad=True)
# optimizer 설정
optimizer = optim.SGD(model.parameters(),lr=0.1)
nb_epochs=1000
for epoch in range (nb_epochs+1):
  # H(x) 계산
  prediction = model(x_train)
  # Cost 계산
  cost = F.cross_entropy(prediction, y_train)
  # Cost로 H(x) 계산
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch,nb_epochs,cost.item()
    ))

Epoch    0/1000 Cost: 2.383857
Epoch  100/1000 Cost: 0.697646
Epoch  200/1000 Cost: 0.521098
Epoch  300/1000 Cost: 0.396014
Epoch  400/1000 Cost: 0.313727
Epoch  500/1000 Cost: 0.281993
Epoch  600/1000 Cost: 0.259930
Epoch  700/1000 Cost: 0.242190
Epoch  800/1000 Cost: 0.227508
Epoch  900/1000 Cost: 0.215087
Epoch 1000/1000 Cost: 0.204393


In [ ]:
# training with low-level cross entropy loss (3)
x_train = [[1,2,1,1],[2,1,3,2],[1,2,2,2],[1,3,3,3],[1,4,4,4],[1,5,5,5],[1,6,6,6],[1,7,7,7]]
y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
# 모델 초기화
W = torch.zeros((4,3),requires_grad=True)
b = torch.zeros(1,requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W,b],lr=0.1)
nb_epochs=1000
for epoch in range (nb_epochs+1):
  # Cost 계산
  z = x_train.matmul(W)+b
  cost = F.cross_entropy(z, y_train)
  # Cost로 H(x) 계산
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch,nb_epochs,cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.739279
Epoch  200/1000 Cost: 0.606837
Epoch  300/1000 Cost: 0.520902
Epoch  400/1000 Cost: 0.455468
Epoch  500/1000 Cost: 0.402059
Epoch  600/1000 Cost: 0.357047
Epoch  700/1000 Cost: 0.321602
Epoch  800/1000 Cost: 0.302932
Epoch  900/1000 Cost: 0.288865
Epoch 1000/1000 Cost: 0.276602
